# Neural Collaborative Filtering vs Matrix Factorization Revisited


https://arxiv.org/pdf/2005.09683.pdf

추천시스템의 방법론 중에 Neural Net을 활용한 Neural Collaborative Filtering(NeuMF)가 전통적인 Matrix Factorization(MF)를 대체 방안이 될 수 있는지를 다양한 선례로 알아보는 Article

## 1. Abstract
전통적으로 dot product나 비슷한 수준을 상위로 정렬하는 방식은 추천시스템에 많이 채용되었다. 그러나 최근에는 이러한 유사성을 Neural Net으로 학습하는 Neural Collaborative Filitering 방식이 등장하고 있다. 해당 Article에서는 NeuMF를 위한 적절한 파라미터, dot product를 학습하기 위한 MLP의 추정함수 그리고 MLP 기반 알고리즘들이 너무 고비용적이고 왜 dot product 기반의 알고리즘을 사용해야 하는지 알아보자. 마지막으로 제목의 질문이 DNN을 기반으로 하는 NeuMF가 더 좋다라는 답변을 내포하지 않는 이유를 살펴본다.

## 1.a. Prequisite; Neural Collaborative Filtering 

https://arxiv.org/pdf/1708.05031.pdf

### reference

* Collaborative Filtering using Deep Neural Networks (in Tensorflow)<br/>
https://medium.com/@victorkohler/collaborative-filtering-using-deep-neural-networks-in-tensorflow-96e5d41a39a1
* Neural Collaborative Filtering 논문 리뷰<br/>
https://dnddnjs.github.io/recomm/2019/08/15/neural_collaborative_filtering/
* Neural Collaborative Filtering with Movie Data<br/>
https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recommender/neural_collaborative_filtering
* Neural Collaborative Filtering 리뷰<br/>
https://itkmj.blogspot.com/2019/09/neural-collaborative-filtering.html
* Neural Collaborative Filtering Implementation<br/>
https://towardsdatascience.com/neural-collaborative-filtering-96cef1009401    

### 1.a.1. Neural Collaborative Filtering Architecture

<div align="center">
<img src = "https://miro.medium.com/max/1400/1*aP-Mx266ExwoWZPSdHtYpA.png" />
</div>

1. Input Layer binarise a sparse vector for a user and item identification where:
Item (i): 1 means the user u has interacted with Item(i)
User (u): To identify the user
2. Embedding layer is a fully connected layer that projects the sparse representation to a dense vector. The obtained user/item embeddings are the latent user/item vectors.
3. Neural CF layers use Multi-layered neural architecture to map the latent vectors to prediction scores.
4. The final output layer returns the predicted score by minimizing the pointwise loss/pairwise loss.

### 1.a.2. Neural Matrix Factorization Architecture

<div align="center">
<img src="imgs/NeuMF_vs_MF_NeuMF_archi.jpg" />
</div>

### setup

NeuMF를 검증하기 위한 데이터는 implicit feedback을 recommendation을 위한 데이터로 사용하였다.

## 2. Definitions
$X: Matrix; x: vectors; x: scalar$

$[x,z]: x와 z벡터의 concatenation$

$\phi: R^d \times R^d\rightarrow R: 두개의 n차원 임베딩 벡터의 결합\ p\in R^d and\ q \in R^d$

p는 user embedding, q는 item embedding

$\phi(p,q): user와\ item의\ 유사도$

<div align="center">
<img src="imgs/NeuMF_vs_MF_simliarity.jpg" />
</div>

### Dot Product (Matrix Factorization)

$\phi^dot(p,q):=<p,q>=p^Tq=\overset{n}{\underset{f=1}{\sum}}p_fq_f$

R: User-Item Matrix
P: User-Latent Factor Matrix
Q: Item-Latent Factor Matrix

희소행렬로 구성된 User-Item Matrix(R)을 임의의 잠재 요인의 개수 L을 선정하여 User-Latent Factor Matrix(P)와 Item-Latent Factor Matrix(Q)로 분리한다. 그리고 두 분리한 P와 Q Matrix를 다시 내적한 결과를 R과 오차값을 통해 조정함으로써 다른 희소행렬의 값을 0이 아닌 값으로 채운다.

$R:\ m\times{n}\ matrix;\ P:\ m\times{l}\ matrix;\ Q:\ n\times{l}\ matrix;\$

1. P와 Q행렬은 1이하의 임의의 실수값으로 random하게 부여한다.
2. P와 Q행렬의 내적을 계산한다.
3. 2의 결과 값을 R행렬의 0이 아닌값과 오차를 계산한다.
4. 오차를 활용하여 P와 Q행렬의 값을 update 한다.
5. 임의의 시행 동안 2~4의 과정을 반복

R 행렬의 임의의 non-zero element i행 j열의 R[i,j]값에 대하여 Update 실시

$P[i]=P[i]+learning\_rate*{e_{ij}*Q[j]-r\_lambda*P[i]}$

$Q[j]=Q[j]+learning\_rate*{e_{ij}*P[i]-r\_lambda*Q[j]}$

In [5]:
import os
import pandas as pd
import numpy as np

path = os.path.join(os.path.dirname(os.path.abspath("__file__")),"data")
movies = pd.read_csv(os.path.join(path,"movies.csv"))
ratings = pd.read_csv(os.path.join(path,"ratings.csv"))
ratings = ratings[["userId","movieId","rating"]]
ratings_matrix = ratings.pivot_table("rating",index="userId",columns="movieId")

In [6]:
rating_movies = pd.merge(ratings,movies,on="movieId")
ratings_matrix = rating_movies.pivot_table("rating",index="userId",columns="title")

In [7]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P, Q, non_zeros):
  actual = []
  preds = []

  for i,j,r in non_zeros:
    actual.append(r)
    preds.append(np.dot(P[i,:], Q[j,:].T))

  result = mean_squared_error(actual,preds)
  return result

In [8]:
def matrix_factorization(R,K,steps=200,learning_rate=0.01,r_lambda = 0.01):
  num_users, num_items = R.shape

  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users,K))
  Q = np.random.normal(scale=1./K, size=(num_items,K))

  prev_rmse = 10000
  break_count = 0

  non_zeros = [ (i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

  for step in range(steps):
    for i,j,r in non_zeros:
      eij = r - np.dot(P[i,:],Q[j,:].T)

      P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R,P, Q, non_zeros)
    if(step % 10) == 0:
      print("### iteration step: ",step," rmse: ",rmse)

  return P, Q

In [9]:
P, Q = matrix_factorization(ratings_matrix.values,K=50,steps=200,learning_rate=0.01,r_lambda=0.01)

### iteration step:  0  rmse:  8.423705034701914
### iteration step:  10  rmse:  0.5381350082096514
### iteration step:  20  rmse:  0.2616873953526066
### iteration step:  30  rmse:  0.13884289422659946
### iteration step:  40  rmse:  0.08766445000765151
### iteration step:  50  rmse:  0.06352180214146708
### iteration step:  60  rmse:  0.05056878035552725
### iteration step:  70  rmse:  0.04278880610647561
### iteration step:  80  rmse:  0.03768808288452511
### iteration step:  90  rmse:  0.03411439291872163
### iteration step:  100  rmse:  0.031481147723579454
### iteration step:  110  rmse:  0.02946450167499212
### iteration step:  120  rmse:  0.027872910023915155
### iteration step:  130  rmse:  0.026586255346641597
### iteration step:  140  rmse:  0.02552546850158907
### iteration step:  150  rmse:  0.024636400788713323
### iteration step:  160  rmse:  0.023880751551717972
### iteration step:  170  rmse:  0.02323069360565542
### iteration step:  180  rmse:  0.02266554560767273
###

In [10]:
pred_matrix = P.dot(Q.T)
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


### Learned Similarity

* $f_{w,b}(x) = \sigma(Wx+b), \sigma(z)=[\sigma(z_1),...,\sigma(z_{out})]$<br/>
$f:R^{d_{in}}\rightarrow R^{d_{out}};\ W\in R^{in\times out};\ b\in R^{out}; \sigma: R\rightarrow R$
* MLP: $\phi^{MLP}(p,q):=f_{W_l,b_l}(...f_{W_1,b_1}([p,q])...)$
* NeuMF: $\phi^{NeuMF}(p,q):=\phi^{MLP}(p_{[1,...,j],q_{[i...j]}}) + \phi^{GMF}(p_{[1,...,j],q_{[i...j]}})$
* GMF: $\phi^{GMF}(p,q):=\sigma(w_T(p\odot{q}))=\sigma(<w\odot{p,q}>)=\sigma(\overset{d}{\underset{f=1}{\sum}}w_fp_fq_f)$

## 3. Revisiting NCF Experiments

Prequsite NeuMF 논문에서 살펴본 것과 같이, 추천을 위한 데이터는 implicit feedback만을 사용하였다. 그리고 추천의 검증을 위해 사용하는 Metric은 Hit ratio(해당 item이 정답 item list에 들어있는 비율)와 NDCG(Rank 추정을 위한 Metric)가 있다.

### NDCG

nDCG를 계산할 때, 정답 안에서의 순서는 무시됩니다. 예를 들어서 곡에 대한 ground truth 가 [1, 2, 3, 4, 5] 이었을 때, 예상 곡 리스트로 [10, 20, 1, 2, 3] 을 제출하든 [10, 20, 2, 3, 1] 를 제출하든, 뒤에 3개가 정답이었기 때문에 두 제출의 점수는 동일합니다. 다만, [1, 2, 3, 10, 20] 을 제출할 경우, nDCG의 정의에 의해 [10, 20, 1, 2, 3] 보다 점수가 높습니다. 채점 코드는 다음과 같으니 참고해 주세요.

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)
        return dcg / self._idcgs[len(gt)]

### Results

<div align="center">
<p><b>Perfomances of Methods</b></p>
<img src="imgs/NeuMF_vs_MF_performance.jpg" />
</div>

#### Matrix Factorization vs MLP
해당 논문에서 보려고 하는 취지와 달리, dot product를 통한 MF가 MLP보다 훨씬 높은 성능을 보인다. 따라서, MLP를 이용한 방식이 MF를 이용한 방식보다 뛰어나다고 말할 수 없다. 그리고 MLP를 이용한 방식은 정확도 뿐만 아니라 model parameter 학습 및 비용적인 측면을 고려했을 때 단점이 보인다.

#### Matrix Factorization vs NeuMF
NeuMF는 dot product와 MLP모델을 동시에 합하여 처리하는 모델이다. 전반적으로 MLP보다는 약간의 성능상승이 있지만 MF보다는 전반적으로 훨씬 좋지 못하다.

그리고 GMF와 MLP를 모두 Fine Tuning후에 합치는 구조는 위의 Tuning을 거치지 않고 합치는 모델보다는 성능이 높다. 그러나 전반적은 MF의 성능보다는 낮다.

그러나 GMF와 MLP를 합치는 것은 도움이 된다는 것을 알 수 있다.

### Futher comparison

<img src="imgs/NeuMF_vs_MF_futher_comparison.jpg" />

## 4. Learning a Dot Product with MLP is Hard

MLP는 dot product를 활용한 방식보다 더 강력한 embedding vector들의 combiner이다. 그러나 이는 MLP에 사용되는 target function의 학습이 얼마나 어려운지를 모르고 하는 말이다.

MLP에서 class function이 클수록 더 많은 파라미터를 요구한다. 더 나아가 학습에 있어서 더 많은 데이터를 요구하여 이러한 target function을 학습하는데 어려움이 있을 것이다.

<div align="center">
<img src="imgs/NeuMF_vs_MF_MLP_perf.jpg"/>
</div>

NeuMF 논문에서 두 user, item의 embedding 벡터를 concat한 2d size의 벡터와 [4h, 2h, h]로 이어지는 3개의 은닉층을 활용하여 신경망 모델을 생성했다. 이 논문에서 은닉층의 뉴런 값을 $h=d/2$로 추천한다. 그리고 embedding vector공간 d에 따라 학습해야 하는 파라미터의 수는 $18d^2$이다(e.g) d=8: 1,152, d=64: 73,728, d=256: 1,179,648).

위의 도식으로, 넓은 수준의 은닉층 공간과 많은 양의 훈련 데이터만 있다면 MLP는 학습이 가능하다는 것을 알 수 있다. 

## 5. Applicability of Dot Product Models

recommendation system은 추천의 결과 만큼 이를 계산하고 도출하는 시간적인 문제 또한 중요하다. 그래서 우리가 시도했던 알고리즘들의 시간복잡도를 살펴보자.

* dot product: $O(d)$ <br/>
: sublinear time algorithm 有
* MLP-learned similarity: $O(d^2)$ <br/>
: sublinear time algorithm 無

만약, 점수를 계산해야하는 n개의 아이템이 있다면, $O(dn)\ vs\ O(d^2n)$의 시간복잡도를 가진다.

거대한 어플리케이션의 경우에 n은 수백만, d는 수백의 값을 가진다. 그럼에도 dot product 방식은 낮은 복잡도로 인해 요구되는 시간내로 유의한 결과를 낼 수 있는 것이다. 그러나 dot product 방식에도 효율적으로 유사한 아이템들을 찾아내야 하는 문제가 있다. 여기에는 'approximate nearest neighbor search'와 'maximum inner product search' 두 가지 방식이 있다.

따라서 실시간 top-N recommendation에 있어서 적합한 방법이 아니다.

## 6. Related Work

### 6.1. Dot products at the Output Layer of DNNs
(<a href="https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf">**'Deep Neural Networks for YouTube Recommendations'**</a> of Candidate Generate Model)

이 방법 또한 Neural Net을 사용하지만 MLP는 아니다. 이는 일반적인 다중 분류 문제로 취급하여 해결한다. input x를 DNN f를 거친 하나의 embedding한 vector로 변환한다.

$input\  x;\ label\ y \in {1,...,n}; DNN:\ f(x)\in{R^d}$

위의 결과로 얻은 vector $R^d$는 vector의 점수를 나타내기 위해 class label을 활용한다. 일반적으로 ㅑinput 결과인 $R^d$를 class matrix $Q\in{R^{n\times{d}}}$와 곱하여 n class들에 대한 스칼라 점수를 계산한다. 이 결과에 대해 softmax cross entrophy를 활용하여 손실을 계산한다.

$Qf(x) = Qp = [<p,q_i>]_{i=1}^{n}$

결국 이는 input embedding과 label embedding을 dot product하여 결과를 얻는것을 말한다.


### 6.2. MLPs at the Output Layer of DNNs

### 6.3. Specialized Structures inside a DNN

## 7. Conclusion

결과적으로는 dot product 방식을 통해 embedding을 combine하는 것이 MLP나 NeuMF 보다 더 나은 선택이라는 것을 알아보았다. dot product방식을 사용하도록 권고하는것은 여러 긍정적인 효과를 가지게 될 것이다.

1. Model의 실제 업무와 관련성 제고
2. Modeling 및 학습에 있어서 단순화를 통해 실행 및 이해를 증진